In [1]:
import pandas as pd

In [2]:
df_forecast = pd.read_parquet("/home2/s5549329/windAI_rug/WindAi/given_datasets/met_forecast.parquet")
df_nowcasting = pd.read_parquet("/home2/s5549329/windAI_rug/WindAi/given_datasets/met_nowcast.parquet")
df_power = pd.read_parquet("/home2/s5549329/windAI_rug/WindAi/given_datasets/wind_power_per_bidzone.parquet")
df_metadata = pd.read_csv("/home2/s5549329/windAI_rug/WindAi/given_datasets/windparks_bidzone.csv")

In [3]:
df_power_long = df_power.reset_index().melt(
    id_vars="index", 
    var_name="bidding_area", 
    value_name="power_MW"
)
df_power_long = df_power_long.rename(columns={"index" : "time"})
df_power_long["bidding_area"] = df_power_long["bidding_area"].str.replace("ELSPOT ", "")

df_power_long.head()

,time,bidding_area,power_MW
0,2020-01-01 00:00:00,NO1,149.285262
1,2020-01-01 01:00:00,NO1,152.634024
2,2020-01-01 02:00:00,NO1,151.163256
3,2020-01-01 03:00:00,NO1,150.223341
4,2020-01-01 04:00:00,NO1,157.415142


In [4]:
start = max(
    df_forecast['time'].min(),
    df_nowcasting.index.min(),
    df_power_long['time'].min()
)

end = min(
    df_forecast['time'].max(),
    df_nowcasting.index.max(),
    df_power_long['time'].max()
)

In [5]:
df_forecast = df_forecast[(df_forecast['time'] >= start) & (df_forecast['time'] <= end)]
df_nowcasting = df_nowcasting[(df_nowcasting.index >= start) & (df_nowcasting.index <= end)]
df_power_long = df_power_long[(df_power_long['time'] >= start) & (df_power_long['time'] <= end)]

In [6]:
meta_set = set(df_metadata["substation_name"])
nowcast_set = set(df_nowcasting["windpark"])
forecast_set = set(df_forecast["sid"])

common = meta_set & nowcast_set & forecast_set
print(f"Common windparks in all datasets: {len(common)}")
print(sorted(common))

to_drop_from_meta = meta_set - common
to_drop_from_nowcast = nowcast_set - common
to_drop_from_forecast = forecast_set - common

print("\n To drop from Meta_data:")
print(sorted(to_drop_from_meta))

print("\n To drop from MET_nowcast:")
print(sorted(to_drop_from_nowcast))

print("\n To drop from MET_forecast:")
print(sorted(to_drop_from_forecast))

Common windparks in all datasets: 59
['Bessakerfjellet', 'Bjerk_VK Vindpark', 'Buheii Vindpark', 'Dønnesfjord Vind', 'Egersund Vindkrv', 'Einarsdalen', 'Engerfjellet', 'Fakken', 'Frøya Vindpark', 'Geitfjellet', 'Gismarvik Vindpark', 'Guleslettene Vindpark', 'Hamnefjell', 'Haraheia', 'Haram Kraft', 'Harbaksfjellet', 'Havøygavlen', 'Hennøy', 'Hitra', 'Hundhammerfjelle', 'Hån Vindpark', 'Høg Jæren', 'Kjølberget', 'Kjøllefjord vindpark', 'Kvenndalsfjellet', 'Kvitfjell vindpark', 'Lista VK', 'Lutelandet', 'Marker Vindpark', 'Mehuken', 'Midtfjellet', 'Måkaknuten', 'Nygårdsfjellet', 'Okla Vindkraftverk', 'Raggovidda', 'Raskiftet', 'Raudfjell Vindpark', 'Sandøy Vindkraft', 'Skomakerfjellet', 'Skudeneshavn', 'Smøla', 'Songkjølen', 'Stokkeland', 'Stokkfjellet', 'Storheia', 'Storøy vindpark', 'Svåheia', 'Sørmarkfjellet', 'Tellenes', 'Tindafjellet', 'Tysvær Vindpark', 'Valsn_Vimle', 'Valsneset', 'Vardafjell', 'Ytre Vikna', 'Ånstadblåheia', 'Øie', 'Øyfjell1', 'Øyfjell2']

 To drop from Meta_data:
[

In [7]:
df_metadata = df_metadata[df_metadata["substation_name"].isin(common)]
df_nowcasting = df_nowcasting[df_nowcasting["windpark"].isin(common)]
df_forecast = df_forecast[df_forecast["sid"].isin(common)]

In [8]:
df_forecast.head()

,sid,time_ref,time,lt,ws10m_00,ws10m_01,ws10m_02,ws10m_03,ws10m_04,ws10m_05,...,g10m_05,g10m_06,g10m_07,g10m_08,g10m_09,g10m_10,g10m_11,g10m_12,g10m_13,g10m_14
0,Engerfjellet,2020-02-15 12:00:00,2020-02-15 12:00:00,0,2.096957,2.110652,2.304629,1.942044,1.447011,2.288181,...,6.918935,8.110546,5.988888,6.934274,5.235212,5.507061,7.193576,6.753938,4.841811,4.875890
1,Engerfjellet,2020-02-15 12:00:00,2020-02-15 13:00:00,1,3.449402,3.436193,3.205872,3.450461,3.143307,4.278822,...,9.914339,9.184751,6.999992,7.892150,7.214608,8.252510,9.535547,8.717684,8.152432,6.240438
2,Engerfjellet,2020-02-15 12:00:00,2020-02-15 14:00:00,2,3.348926,2.903557,2.779971,3.733802,3.148597,2.912592,...,10.031385,8.624619,7.984353,8.264300,7.205638,9.052546,9.531441,8.367238,7.840395,7.153080
3,Engerfjellet,2020-02-15 12:00:00,2020-02-15 15:00:00,3,3.095971,2.560161,2.448126,2.267286,3.050008,3.310141,...,8.185250,8.821701,7.481009,8.172009,6.255360,7.345034,8.303812,6.735660,7.600200,7.189164
4,Engerfjellet,2020-02-15 12:00:00,2020-02-15 16:00:00,4,2.821545,2.965743,3.235484,3.212730,3.064931,2.420390,...,7.551644,9.709259,8.365305,8.703517,6.151643,10.116440,7.346225,9.079601,9.048587,6.937930


In [9]:
df_forecast.groupby(["sid", "time"]).size().value_counts()

3    1469749
2    1080585
1      78411
dtype: int64

In [10]:
forecast_cols_w_speed = [col for col in df_forecast.columns if "ws10m_" in col]
forecast_cols_w_direction = [col for col in df_forecast.columns if "wd10m_" in col]
forecast_cols_t = [col for col in df_forecast.columns if "t2m_" in col]
forecast_cols_rh = [col for col in df_forecast.columns if "rh2m_" in col]
forecast_cols_mslp = [col for col in df_forecast.columns if "mslp_" in col]
forecast_cols_g = [col for col in df_forecast.columns if "g10m_" in col]
df_forecast["ws10m_mean"]  = df_forecast[forecast_cols_w_speed].mean(axis=1)
df_forecast["ws10m_std"]   = df_forecast[forecast_cols_w_speed].std(axis=1)

df_forecast["wd10m_mean"]  = df_forecast[forecast_cols_w_direction].mean(axis=1)
df_forecast["wd10m_std"]   = df_forecast[forecast_cols_w_direction].std(axis=1)

df_forecast["t2m_mean"]    = df_forecast[forecast_cols_t].mean(axis=1)
df_forecast["t2m_std"]     = df_forecast[forecast_cols_t].std(axis=1)

df_forecast["rh2m_mean"]   = df_forecast[forecast_cols_rh].mean(axis=1)
df_forecast["rh2m_std"]    = df_forecast[forecast_cols_rh].std(axis=1)

df_forecast["mslp_mean"]   = df_forecast[forecast_cols_mslp].mean(axis=1)
df_forecast["mslp_std"]    = df_forecast[forecast_cols_mslp].std(axis=1)

df_forecast["g10m_mean"]   = df_forecast[forecast_cols_g].mean(axis=1)
df_forecast["g10m_std"]    = df_forecast[forecast_cols_g].std(axis=1)

summary_cols = [
    "ws10m_mean", "ws10m_std",
    "wd10m_mean", "wd10m_std",
    "t2m_mean",   "t2m_std",
    "rh2m_mean",  "rh2m_std",
    "mslp_mean",  "mslp_std",
    "g10m_mean",  "g10m_std"
]

df_forecast_avg = df_forecast.groupby(["sid", "time"])[summary_cols].mean().reset_index()

In [11]:
df_forecast_avg["time"] = pd.to_datetime(df_forecast_avg["time"])
df_forecast_avg.head()

,sid,time,ws10m_mean,ws10m_std,wd10m_mean,wd10m_std,t2m_mean,t2m_std,rh2m_mean,rh2m_std,mslp_mean,mslp_std,g10m_mean,g10m_std
0,Bessakerfjellet,2020-02-15 12:00:00,7.294972,1.315213,235.690861,6.591162,277.660277,0.497662,0.860875,0.055132,99775.850667,44.163305,14.636166,1.523008
1,Bessakerfjellet,2020-02-15 13:00:00,6.639898,0.825916,227.075603,6.880623,277.617520,0.535993,0.876358,0.041548,99816.510933,41.808119,13.252052,1.818680
2,Bessakerfjellet,2020-02-15 14:00:00,5.024071,0.778648,216.288069,16.480253,277.159103,0.600454,0.876495,0.033818,99825.021933,48.331877,11.903856,1.589320
3,Bessakerfjellet,2020-02-15 15:00:00,3.518165,0.369989,174.336054,16.726134,276.441382,0.502003,0.871451,0.037151,99804.551333,45.440402,8.558784,1.124682
4,Bessakerfjellet,2020-02-15 16:00:00,4.532025,0.414117,140.536212,6.787815,275.231637,0.345054,0.870454,0.027691,99732.513333,53.689166,7.365164,0.674284


In [12]:
df_forecast_avg.columns

Index(['sid', 'time', 'ws10m_mean', 'ws10m_std', 'wd10m_mean', 'wd10m_std',
       't2m_mean', 't2m_std', 'rh2m_mean', 'rh2m_std', 'mslp_mean', 'mslp_std',
       'g10m_mean', 'g10m_std'],
      dtype='object')

In [13]:
df_nowcasting = df_nowcasting.rename(columns={"windpark": "sid"})
df_nowcasting = df_nowcasting.reset_index()
df_nowcasting["time"] = pd.to_datetime(df_nowcasting["time"])
df_nowcasting.head()

,time,air_temperature_2m,air_pressure_at_sea_level,relative_humidity_2m,precipitation_amount,wind_speed_10m,wind_direction_10m,sid
0,2020-02-15 12:00:00,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506,Bessakerfjellet
1,2020-02-15 13:00:00,276.076935,99832.359375,0.891700,0.000000e+00,6.657547,233.088379,Bessakerfjellet
2,2020-02-15 14:00:00,275.843689,99830.406250,0.881385,9.209055e-05,5.876413,229.860733,Bessakerfjellet
3,2020-02-15 15:00:00,274.957886,99853.375000,0.837362,0.000000e+00,3.329724,203.135254,Bessakerfjellet
4,2020-02-15 16:00:00,273.522888,99767.031250,0.812765,8.627482e-15,3.800424,142.939713,Bessakerfjellet


In [14]:
df_nowcasting = df_nowcasting.rename(columns={
    "air_temperature_2m": "t2m_now",
    "air_pressure_at_sea_level": "mslp_now",
    "relative_humidity_2m": "rh2m_now",
    "wind_speed_10m": "ws10m_now",
    "wind_direction_10m": "wd10m_now",
    "precipitation_amount": "precip_now",
})

In [15]:
df_nowcasting.head()

,time,t2m_now,mslp_now,rh2m_now,precip_now,ws10m_now,wd10m_now,sid
0,2020-02-15 12:00:00,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506,Bessakerfjellet
1,2020-02-15 13:00:00,276.076935,99832.359375,0.891700,0.000000e+00,6.657547,233.088379,Bessakerfjellet
2,2020-02-15 14:00:00,275.843689,99830.406250,0.881385,9.209055e-05,5.876413,229.860733,Bessakerfjellet
3,2020-02-15 15:00:00,274.957886,99853.375000,0.837362,0.000000e+00,3.329724,203.135254,Bessakerfjellet
4,2020-02-15 16:00:00,273.522888,99767.031250,0.812765,8.627482e-15,3.800424,142.939713,Bessakerfjellet


In [17]:
print(df_forecast_avg["time"].min(), df_forecast_avg["time"].max())


2020-02-15 12:00:00 2025-03-27 01:00:00


In [18]:
df_merged_weather = pd.merge(df_forecast_avg, df_nowcasting, on=["sid", "time"], how="inner")
df_merged_weather.head()

,sid,time,ws10m_mean,ws10m_std,wd10m_mean,wd10m_std,t2m_mean,t2m_std,rh2m_mean,rh2m_std,mslp_mean,mslp_std,g10m_mean,g10m_std,t2m_now,mslp_now,rh2m_now,precip_now,ws10m_now,wd10m_now
0,Bessakerfjellet,2020-02-15 12:00:00,7.294972,1.315213,235.690861,6.591162,277.660277,0.497662,0.860875,0.055132,99775.850667,44.163305,14.636166,1.523008,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506
1,Bessakerfjellet,2020-02-15 13:00:00,6.639898,0.825916,227.075603,6.880623,277.617520,0.535993,0.876358,0.041548,99816.510933,41.808119,13.252052,1.818680,276.076935,99832.359375,0.891700,0.000000e+00,6.657547,233.088379
2,Bessakerfjellet,2020-02-15 14:00:00,5.024071,0.778648,216.288069,16.480253,277.159103,0.600454,0.876495,0.033818,99825.021933,48.331877,11.903856,1.589320,275.843689,99830.406250,0.881385,9.209055e-05,5.876413,229.860733
3,Bessakerfjellet,2020-02-15 15:00:00,3.518165,0.369989,174.336054,16.726134,276.441382,0.502003,0.871451,0.037151,99804.551333,45.440402,8.558784,1.124682,274.957886,99853.375000,0.837362,0.000000e+00,3.329724,203.135254
4,Bessakerfjellet,2020-02-15 16:00:00,4.532025,0.414117,140.536212,6.787815,275.231637,0.345054,0.870454,0.027691,99732.513333,53.689166,7.365164,0.674284,273.522888,99767.031250,0.812765,8.627482e-15,3.800424,142.939713


In [19]:
df_merged_weather.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ws10m_mean,2628686.0,5.042511,3.080494,2.163493e-01,2.711206,4.314413,6.701969,27.741219
ws10m_std,2628686.0,1.028123,0.569205,2.620698e-02,0.627330,0.902428,1.287740,8.003559
wd10m_mean,2628686.0,187.460442,78.022339,2.893837e+00,128.769440,184.671647,248.556912,357.572138
wd10m_std,2628686.0,31.785038,35.899724,4.788954e-01,7.993889,16.101267,41.576451,184.855365
t2m_mean,2628686.0,278.475022,7.032232,2.495424e+02,273.688618,278.244952,283.657917,304.872186
t2m_std,2628686.0,0.738235,0.422825,6.846008e-02,0.451539,0.637220,0.905126,5.786043
rh2m_mean,2628686.0,0.813164,0.132957,1.624328e-01,0.732471,0.835163,0.918755,1.000007
rh2m_std,2628686.0,0.049903,0.027279,1.559720e-07,0.030287,0.045695,0.065304,0.264658
mslp_mean,2628686.0,101025.466478,1263.913994,9.452204e+04,100229.624692,101115.302739,101894.767117,105103.114833
mslp_std,2628686.0,103.821030,53.100382,1.418981e+01,66.757817,90.732860,126.816770,834.418348


In [20]:
df_metadata = df_metadata.rename(columns={"substation_name": "sid"})
df_metadata["bidding_area"] = df_metadata["bidding_area"].str.replace("ELSPOT ", "")
df_metadata.head()

,bidding_area,sid,operating_power_max,prod_start_new,eic_code
0,NO1,Engerfjellet,52.8,2022-09-19 13:00:00,50WP00000002158R
1,NO1,Hån Vindpark,21.0,2022-10-26 22:00:00,50WP000000022251
2,NO1,Kjølberget,55.9,2020-07-07 09:00:00,50WP00000002085S
3,NO1,Marker Vindpark,54.0,2020-01-01 00:00:00,50WP00000002048Y
4,NO1,Raskiftet,111.6,2020-01-01 00:00:00,50WP00000001718K


In [21]:
df_power_long.head()

,time,bidding_area,power_MW
1056,2020-02-15 12:00:00,NO1,72.052233
1057,2020-02-15 13:00:00,NO1,44.666173
1058,2020-02-15 14:00:00,NO1,65.059451
1059,2020-02-15 15:00:00,NO1,86.562093
1060,2020-02-15 16:00:00,NO1,77.712180


In [22]:
df_power_long_sid = pd.merge(
    df_power_long,
    df_metadata[["bidding_area", "sid"]],
    on="bidding_area",
    how="left"
)
df_power_long_sid.head()

,time,bidding_area,power_MW,sid
0,2020-02-15 12:00:00,NO1,72.052233,Engerfjellet
1,2020-02-15 12:00:00,NO1,72.052233,Hån Vindpark
2,2020-02-15 12:00:00,NO1,72.052233,Kjølberget
3,2020-02-15 12:00:00,NO1,72.052233,Marker Vindpark
4,2020-02-15 12:00:00,NO1,72.052233,Raskiftet


In [24]:
df_final_sid = pd.merge(
    df_merged_weather,
    df_power_long_sid[["time", "sid", "power_MW", "bidding_area"]],
    on=["time", "sid"],
    how="inner"
)

df_final_sid.head()

,sid,time,ws10m_mean,ws10m_std,wd10m_mean,wd10m_std,t2m_mean,t2m_std,rh2m_mean,rh2m_std,...,g10m_mean,g10m_std,t2m_now,mslp_now,rh2m_now,precip_now,ws10m_now,wd10m_now,power_MW,bidding_area
0,Bessakerfjellet,2020-02-15 12:00:00,7.294972,1.315213,235.690861,6.591162,277.660277,0.497662,0.860875,0.055132,...,14.636166,1.523008,276.335876,99795.664062,0.821845,2.238679e-04,8.104369,246.912506,588.636703,NO3
1,Bessakerfjellet,2020-02-15 13:00:00,6.639898,0.825916,227.075603,6.880623,277.617520,0.535993,0.876358,0.041548,...,13.252052,1.818680,276.076935,99832.359375,0.891700,0.000000e+00,6.657547,233.088379,460.424020,NO3
2,Bessakerfjellet,2020-02-15 14:00:00,5.024071,0.778648,216.288069,16.480253,277.159103,0.600454,0.876495,0.033818,...,11.903856,1.589320,275.843689,99830.406250,0.881385,9.209055e-05,5.876413,229.860733,297.167215,NO3
3,Bessakerfjellet,2020-02-15 15:00:00,3.518165,0.369989,174.336054,16.726134,276.441382,0.502003,0.871451,0.037151,...,8.558784,1.124682,274.957886,99853.375000,0.837362,0.000000e+00,3.329724,203.135254,314.183239,NO3
4,Bessakerfjellet,2020-02-15 16:00:00,4.532025,0.414117,140.536212,6.787815,275.231637,0.345054,0.870454,0.027691,...,7.365164,0.674284,273.522888,99767.031250,0.812765,8.627482e-15,3.800424,142.939713,488.289472,NO3


In [32]:
regions = ['NO1', 'NO2', 'NO3', 'NO4']

dfs_by_region = {
    region: df_final_sid[df_final_sid['bidding_area'] == region].copy()
    for region in regions
}

# Optionally assign each to a variable
df_NO1 = dfs_by_region['NO1']
df_NO2 = dfs_by_region['NO2']
df_NO3 = dfs_by_region['NO3']
df_NO4 = dfs_by_region['NO4']

In [33]:
df_NO1.head()

,sid,time,ws10m_mean,ws10m_std,wd10m_mean,wd10m_std,t2m_mean,t2m_std,rh2m_mean,rh2m_std,...,g10m_mean,g10m_std,t2m_now,mslp_now,rh2m_now,precip_now,ws10m_now,wd10m_now,power_MW,bidding_area
311752,Engerfjellet,2020-02-15 12:00:00,2.139608,0.448171,197.466405,11.766175,276.379494,0.395486,0.823404,0.051589,...,6.050247,1.032772,275.696960,100518.625000,0.844108,0.000000e+00,2.051558,193.094101,72.052233,NO1
311753,Engerfjellet,2020-02-15 13:00:00,3.384011,0.476801,207.005511,6.122282,276.422947,0.375587,0.825629,0.047831,...,8.121981,0.954943,275.594910,100569.164062,0.846076,1.011751e-12,4.104134,213.308594,44.666173,NO1
311754,Engerfjellet,2020-02-15 14:00:00,3.116065,0.380970,197.767234,7.315636,276.095935,0.374416,0.847620,0.059136,...,8.276285,0.832413,275.363647,100541.710938,0.870398,1.087279e-03,4.097149,206.835739,65.059451,NO1
311755,Engerfjellet,2020-02-15 15:00:00,2.856627,0.388535,186.196795,7.583082,275.603674,0.339881,0.886108,0.062451,...,7.532237,0.792371,275.035309,100471.039062,0.908804,0.000000e+00,2.759731,192.723495,86.562093,NO1
311756,Engerfjellet,2020-02-15 16:00:00,3.303811,0.599790,185.264509,4.739498,274.942599,0.471217,0.942954,0.047610,...,7.929002,1.163573,274.119812,100361.812500,0.968054,0.000000e+00,2.770049,185.444046,77.712180,NO1


In [34]:
df_final_sid_sorted = df_NO1.sort_values("time")

df_final_sid_sorted = df_final_sid_sorted.reset_index(drop=True)

df_final_sid_sorted.head()

,sid,time,ws10m_mean,ws10m_std,wd10m_mean,wd10m_std,t2m_mean,t2m_std,rh2m_mean,rh2m_std,...,g10m_mean,g10m_std,t2m_now,mslp_now,rh2m_now,precip_now,ws10m_now,wd10m_now,power_MW,bidding_area
0,Engerfjellet,2020-02-15 12:00:00,2.139608,0.448171,197.466405,11.766175,276.379494,0.395486,0.823404,0.051589,...,6.050247,1.032772,275.696960,100518.625000,0.844108,0.000000e+00,2.051558,193.094101,72.052233,NO1
1,Marker Vindpark,2020-02-15 12:00:00,3.623393,0.597573,221.412773,5.837665,278.827723,0.408630,0.861682,0.062364,...,9.070228,1.157618,278.657776,100649.882812,0.858711,4.665708e-04,3.400307,214.018539,72.052233,NO1
2,Kjølberget,2020-02-15 12:00:00,2.844588,0.641755,241.523584,8.134762,274.222099,0.391442,0.832552,0.037155,...,8.612595,0.790866,274.429047,100517.117188,0.852363,5.648359e-05,2.650362,241.739349,72.052233,NO1
3,Raskiftet,2020-02-15 12:00:00,3.048176,0.668024,223.848515,6.317409,273.155895,0.517255,0.855875,0.041512,...,8.389608,0.620932,274.314148,100501.562500,0.871207,1.548093e-11,2.429872,225.234009,72.052233,NO1
4,Songkjølen,2020-02-15 12:00:00,2.607449,0.459566,209.322322,6.382720,276.276455,0.481296,0.857556,0.053489,...,6.645694,0.842453,276.178680,100547.359375,0.865516,5.105352e-04,2.676929,201.159042,72.052233,NO1


In [36]:
df_aggregated = df_NO1.groupby("time").mean().reset_index()

In [37]:
df_final_sid_sorted = df_aggregated.sort_values("time")

df_final_sid_sorted = df_final_sid_sorted.reset_index(drop=True)

df_final_sid_sorted.head()

,time,ws10m_mean,ws10m_std,wd10m_mean,wd10m_std,t2m_mean,t2m_std,rh2m_mean,rh2m_std,mslp_mean,mslp_std,g10m_mean,g10m_std,t2m_now,mslp_now,rh2m_now,precip_now,ws10m_now,wd10m_now,power_MW
0,2020-02-15 12:00:00,2.970600,0.562880,219.089599,7.411888,276.282250,0.433070,0.848453,0.051379,100543.824778,40.157405,7.942286,0.929867,276.305115,100563.515625,0.858436,0.000186,2.752213,214.877258,72.052233
1,2020-02-15 13:00:00,3.358135,0.378822,218.735674,5.881298,276.304744,0.411686,0.842491,0.051295,100579.730611,31.462750,8.381703,0.991343,276.314178,100598.023438,0.852978,0.000617,3.711130,217.242462,44.666173
2,2020-02-15 14:00:00,3.122371,0.452259,206.372581,6.159813,275.975155,0.384912,0.867678,0.055230,100564.250167,44.371400,8.208556,0.789677,275.946930,100593.203125,0.884543,0.000181,3.483180,207.492828,65.059451
3,2020-02-15 15:00:00,2.897094,0.368524,191.985830,6.705309,275.442468,0.356906,0.911628,0.054637,100517.015378,55.889691,7.577428,0.771540,275.529144,100512.468750,0.918824,0.000038,2.808272,196.660538,86.562093
4,2020-02-15 16:00:00,3.035758,0.494459,183.045830,6.086777,274.831068,0.454263,0.958786,0.038495,100458.559889,63.529883,7.640261,0.916110,274.937225,100429.398438,0.980287,0.004540,2.647012,183.282181,77.712180
